In [1]:
# Model Settings
MODEL_NAME="all-MiniLM-L6-v2"

# Pinecone Settings
PINECONE_API_KEY="e72e0b7c-e243-41eb-b497-3fc23b3105fb"
PINECONE_INDEX_NAME="docs"
PINECONE_REGION="us-east1-gcp"

In [2]:
import pinecone
from sentence_transformers import SentenceTransformer

C:\Users\ambar\AppData\Roaming\Python\Python39\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# PINECONE 

In [3]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_REGION)
index = pinecone.Index(PINECONE_INDEX_NAME)

In [4]:
model = SentenceTransformer(MODEL_NAME)

In [5]:
query = "What is Segmentation?"

xq = model.encode([query]).tolist()

In [6]:
xc = index.query(xq, top_k=3,
                 include_metadata=True)

In [7]:
xc

{'matches': [{'id': '31659e4f-f37e-45b9-97fe-bb7ab6a32eb0',
              'metadata': {'category': 'ncert',
                           'filename': 'C:/Ambarish/PINECONE/docs/CHAP06AnatomyFloweringPlants.pdf',
                           'sentence': 'Study of internal structure of plants '
                                       'is called anatomy.'},
              'score': 0.33593443,
              'values': []},
             {'id': '6dccce3d-86ad-4c37-8662-080038157000',
              'metadata': {'category': 'ncert',
                           'filename': 'C:/Ambarish/PINECONE/docs/CHAP04AnimalKingdom.pdf',
                           'sentence': 'Here\n'
                                       'the cells performing the same function '
                                       'are arranged into tissues, hence is\n'
                                       'called tissue level  of organisation.'},
              'score': 0.328107536,
              'values': []},
             {'id': '8ac4ee1b-f

In [8]:
contexts = [
        x['metadata']['sentence'] for x in xc['matches']
    ]

In [9]:
contexts

['Study of internal structure of plants is called anatomy.',
 'Here\nthe cells performing the same function are arranged into tissues, hence is\ncalled tissue level  of organisation.',
 'These features are used\nas the basis of animal classification and some of them are discussed here.']

In [10]:
import openai
key = 'd6bcc40e68fc4119abcd43b4661dc8e3'
location = 'eastus'
endpoint = 'https://openaidemos007.openai.azure.com/'
openai.api_type = "azure"
openai.api_key = key
openai.api_base = endpoint
deployment_id_gpt4='gpt4'
openai.api_key = key

def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"


def generate_answer(conversation):
    openai.api_version = "2023-03-15-preview"
    response = openai.ChatCompletion.create(
    engine=deployment_id_gpt4,
    messages=conversation,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response['choices'][0]['message']['content']).strip()


In [11]:
conversation=[{"role": "system", "content": "Assistant is a large language model trained by OpenAI."}]
context= "\n\n".join(contexts)
prompt = create_prompt(context,query)            
conversation.append({"role": "assistant", "content": prompt})
conversation.append({"role": "user", "content": query})
reply = generate_answer(conversation)

In [12]:
context

'Study of internal structure of plants is called anatomy.\n\nHere\nthe cells performing the same function are arranged into tissues, hence is\ncalled tissue level  of organisation.\n\nThese features are used\nas the basis of animal classification and some of them are discussed here.'

In [13]:
reply

"Segmentation refers to the division of an organism's body into a series of similar, repeating units or segments. This characteristic is commonly found in certain animal groups, such as annelids (segmented worms) and arthropods (insects, spiders, and crustaceans). Segmentation can provide advantages for movement, flexibility, and the ability to adapt to different environments."

In [14]:
from pprint import pprint
pprint(reply)

("Segmentation refers to the division of an organism's body into a series of "
 'similar, repeating units or segments. This characteristic is commonly found '
 'in certain animal groups, such as annelids (segmented worms) and arthropods '
 '(insects, spiders, and crustaceans). Segmentation can provide advantages for '
 'movement, flexibility, and the ability to adapt to different environments.')
